# Refinitiv Data Library for Python
## Define and Open a session

Depending on the Credentials and Refinitiv Products/Services you have access to, you need define & open either a PlatformSession or DesktopSession: 

 - **PlatformSession:** Used to connect to the Refinitiv Data Platform (RDP) directly or via a local (deployed) Real-time Distribution System(RTDS). The PlatformSession requires RDP credentials (machine account) or connection details of your local platform (IP of the local server and DACS user name).
  - When the PlatformSession is used with RDP credentials only, it retrieves both streaming data and non-streaming data from the Refinitiv Data Platform.
  - When the PlatformSession is used with connection details of your local deployed platform, it retrieves streaming data only from your local platform.  
   
   
 - **DesktopSession:** This type of session is used to connect RDP either via Eikon or via the Refinitiv Workspace. Eikon or the Refinitiv Workspace must be run **on the same PC** as your Python code.
 
You will also require an App Key that uniquely identifies your application. Please refer to the Refinitiv Data Libraries Quick Start guide's **Access Credentials** section for instructions on creating an App Key.

## Import the library

In [1]:
import refinitiv.data as rd

## Using the 'refinitiv-data.config.json' file
The Refinitiv data library config file has a default name of **'refinitiv-data.config.json'** allows you to specify session related parameters such as:
* the default session to use
* credentials
* connectivity parameters  

for the various session types.   
Using the config file, where possible, provides the simplest approach to creating and opening sessions.  
For example, assuming you have specified a 'default' session in the config file, you can create and open the default session as follows:

In [ ]:
rd.open_session()

The above will look for an entry such as:  
>    *"sessions": {  
>        "default": "desktop.default",  
>        ....  
>    }*  

and then use the parameters defined in the config section - in this example *"desktop.default"*, to create and open a desktop session.  
Please refer to the supplied example **'refinitiv-data.config.json'** file for the full config example.

### Open a specific configured session
If you have multiple sessions configurations specified in the json file you can open a specific one by name e.g.:

In [ ]:
rd.open_session(name = 'platform.deployed')

The above would open a session based on the parameters specified in the '*"platform.deployed"* section of the json file.

#### Open a session defined in a custom configuration file 
As well as the default config filename of **'refinitiv-data.config.json'**, you can also use a custom file to specify the config.   
The following call opens the default session as defined in your custom configuration file - with the filename passed as a parameter.

In [ ]:
rd.open_session(config_name="refinitiv-data.custom.config.json")

<br>      
Alternatively, if you do not wish to use the refinitiv-data config file, you can define sessions by specifying the parameters and session type explicitly - as shown below.

## Define the session of your choice
The following example calls use credentials/connectivity parameters as defined in the **Configuration//credentials.ipynb** file.     

Please edit **Configuration/credentials.ipynb** to set your credentials and run the next cell before continuing

In [ ]:
%run ../Configuration/credentials.ipynb

#### Desktop Session

Create and open a Desktop session to Refintiv Eikon 4 or Refinitiv Workspace application - running locally (using the **APP_KEY** defined in the above notebook).

In [2]:
session = rd.session.desktop.Definition(app_key = APP_KEY).get_session()

#### OR Platform Session
Create and open a Platform session to connect directly to the Refinitiv Data Platform. 

In [ ]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
).get_session()

#### OR Local(Deployed) Session
Create and open a Platform session to connect to a local (deployed) real-time distribution system(RTDS). 

In [ ]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    deployed_platform_host = DEPLOYED_PLATFORM_HOST,
    deployed_platform_username = DEPLOYED_PLATFORM_USER_NAME
).get_session()

#### OR Local + Platform

Create and open a Platform session to both the Data Platform and a deployed real-time distribution system (RTDS).  
With this configuration, streaming data is retrieved from the local platform while non-streaming data is retrieved from the Refinitiv Data Platform.  
This scenario could apply where you want to consume both internally published streaming data and non-streaming data from Refinitiv

In [ ]:
session = rd.session.platform.Definition(
    app_key = APP_KEY, 
    grant = rd.session.platform.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    ),
    deployed_platform_host = DEPLOYED_PLATFORM_HOST,
    deployed_platform_username = DEPLOYED_PLATFORM_USER_NAME
).get_session()

## Then open it

In [ ]:
session.open()

## Diagnose any Session issues
If you cannot open a session, to generate addtional logging, enable DEBUG logging and call session.open() again

In [ ]:
import logging.config
session.set_log_level(logging.DEBUG)
session.open()
# To set logging back to default value
session.set_log_level(logging.WARNING)

## Close the session

In [4]:
session.close()

<SessionState.Closed: 1>

## Session Events and State change notification
If you want to be notified of any Session events or change in State you can optionally specify State and Event callbacks

In [ ]:
# Callback to handle session state changes
def on_state_cb(session, state_code, state_msg):
    print(f"Session State:{state_code} | {state_msg}")
# Callback to handle session events
def on_event_cb(session, event_code, event_msg):
    print(f"Session event:{event_code} | {event_msg}")

# Using PlatformSession - can be applied to DesktopSession as well
session = rd.session.platform.Definition(
    app_key=APP_KEY,
    grant=rd.session.platform.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
).get_session()

# Set your callbacks for handing the session events
session.on_event(on_event_cb)
session.on_state(on_state_cb)

# Open the Session
session.open()

#
# Run your application code
#


In [6]:
# close the session when done
session.close()